Imports required to perform the extraction operation:

In [3]:
import pandas as pd
from libs.data_helpers import load_song_data

The below extension needs to be activated to show the progress bar

In [4]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
PATH_R = "../MillionSongSubset"
PATH_HDF5 = "../sample_data/TRAXLZU12903D05F94.h5"


In [4]:
df = load_song_data(dataset_root_dir=PATH_R, sample_hdf5_file_path=PATH_HDF5, letter="*", half=None, max_songs=None)

  0%|          | 0/3813 [00:00<?, ?it/s]

Basic information about the df:

In [5]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 54 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   analysis_sample_rate        636 non-null    int32  
 1   artist_7digitalid           636 non-null    int32  
 2   artist_familiarity          636 non-null    float64
 3   artist_hotttnesss           636 non-null    float64
 4   artist_id                   636 non-null    object 
 5   artist_latitude             636 non-null    float64
 6   artist_location             636 non-null    object 
 7   artist_longitude            636 non-null    float64
 8   artist_mbid                 636 non-null    object 
 9   artist_mbtags               636 non-null    object 
 10  artist_mbtags_count         636 non-null    object 
 11  artist_name                 636 non-null    object 
 12  artist_playmeid             636 non-null    int32  
 13  artist_terms                636 non

If we want to extract the df as a compressed json:

In [21]:
df.to_json("20kSongs.json.gz", orient="records", compression={'method': 'gzip', 'compresslevel': 9, 'mtime': 1})

If we want to extract the df to a pickle:

In [16]:
df.to_pickle("20kSongs.pickle.gz", compression={'method': 'gzip', 'compresslevel': 9, 'mtime': 1})

Test whether we can read the pickle we've just created:

In [17]:
df_extracted = pd.read_pickle("20kSongs.pickle.gz", compression={'method': 'gzip', 'compresslevel': 9, 'mtime': 1})